### Libraries import

In [11]:
import operator
import numpy as np
import scipy as sp
import pandas as pd
from tqdm import tqdm
import plotly.io as pio
import statsmodels.api as sm
import plotly.graph_objects as go
import sklearn.metrics as metrics
import sklearn.model_selection as modsel

import warnings
warnings.filterwarnings("ignore")
pio.templates.default = "plotly_dark"

### Functions

In [12]:
def roc_metric(Y, 
               Y_pred):
    """
    Function for the calculation of AUC metric

    Inputs:
    ----------
    Y : DataFrame
        Set of Y for the model
    Y_pred : DataFrame
        Set of predicted Y for the model

    Returns:
    ----------
    auc : float
        AUC for the given series
    thresholds[optimal_index] : float
        Optimal threshold with highest (TPR - FPR)
    """

    fpr, tpr, thresholds = metrics.roc_curve(Y, Y_pred, pos_label=1)
    auc = round(metrics.auc(fpr, tpr), 3)
    optimal_index = np.argmax(tpr - fpr)

    return auc, thresholds[optimal_index]

#---------------------------------------------------------------------------------------

def remove_most_insignificant(X, 
                              X_test, 
                              results):
    """
    Function for the removal of the most insignificant variables from the model

    Inputs:
    ----------
    X : DataFrame
        Set of X for the model
    results : model
        Fitted statsmodels model

    Returns:
    ----------
    X : DataFrame
        Optimized set of X for the validation of the model
    X_test : DataFrame
        Optimized set of X for the testing of the model
    """
    
    # Use operator to find the key which belongs to the maximum value in the dictionary
    max_p_value = max(results.pvalues.iteritems(), key = operator.itemgetter(1))[0]
    # Drop the worst feature
    X.drop(columns = max_p_value, inplace = True)
    X_test.drop(columns = max_p_value, inplace = True)

    return X, X_test

#---------------------------------------------------------------------------------------

def model_optimization(Y_train,
                       Y_test,
                       X_train,
                       X_test,
                       type:str = 'Probit', 
                       p_value_bord:float = 0.05, 
                       silent:bool = False):
    """
    Function for the optimization of OLS

    Inputs:
    ----------
    Y : array
        Target variable for the regression
    X : DataFrame
        Set of X for the model
    type : str = 'Probit'
        Type of the model
    p_value_bord : float = 0.05
        Maximum acceptable p-value for the coefficient
    silent : bool = False
        Whether not to show reports about model

    Returns:
    ----------
    results : model
        Fitted statsmodels model
    auc_train : float
        AUC on the train data
    auc_test : float
        AUC on the test data
    ks_train.pvalue : float
        KS-test p-value on the train data
    ks_test.pvalue : float
        KS-test p-value on the test data
    f1_train : float
        F1-score on the train data
    f1_test : float
        F1-score on the test data
    pr_train : float
        Precision score on the train data
    pr_test : float
        Precision score on the test data
    rec_train : float
        Recall score on the train data
    rec_test : float
        Recall score on the test data
    """
    
    insignificant_feature = True
    while insignificant_feature:
        # Create model
        if type == 'Probit':
            model = sm.Probit(Y_train, X_train)
        else:
            model = sm.Logit(Y_train, X_train)

        # Fit model and get
        results = model.fit(disp = 0)
        significant = [p_value < p_value_bord for p_value in results.pvalues]
        if all(significant):
            insignificant_feature = False
        else:
            # If there's only one insignificant variable left
            if X_train.shape[1] == 1:
                print('No significant features found')
                results = None
                insignificant_feature = False
            else:
                X_train, X_test = remove_most_insignificant(X_train, X_test, results)
    
    Y_train_pred = results.predict(X_train)
    Y_test_pred = results.predict(X_test)
    auc_train, threshold_train = roc_metric(Y_train, Y_train_pred)
    auc_test, threshold_test = roc_metric(Y_test, Y_test_pred)
    Y_train_pred_round = np.where(Y_train_pred < threshold_train, np.floor(Y_train_pred), np.ceil(Y_train_pred))
    Y_test_pred_round = np.where(Y_test_pred < threshold_test, np.floor(Y_test_pred), np.ceil(Y_test_pred))

    ks_samples_train = pd.DataFrame({'Y': Y_train, 'Y_pred': Y_train_pred})
    ks_samples_train_posi = ks_samples_train[ks_samples_train['Y'] == 1]['Y_pred']
    ks_samples_train_nega = ks_samples_train[ks_samples_train['Y'] == 0]['Y_pred']
    ks_train = sp.stats.kstest(ks_samples_train_posi, ks_samples_train_nega)
    ks_samples_test = pd.DataFrame({'Y': Y_test, 'Y_pred': Y_test_pred})
    ks_samples_test_posi = ks_samples_test[ks_samples_test['Y'] == 1]['Y_pred']
    ks_samples_test_nega = ks_samples_test[ks_samples_test['Y'] == 0]['Y_pred']
    ks_test = sp.stats.kstest(ks_samples_test_posi, ks_samples_test_nega)

    f1_train = round(metrics.f1_score(Y_train, Y_train_pred_round), 3)
    f1_test = round(metrics.f1_score(Y_test, Y_test_pred_round), 3)
    pr_train = round(metrics.precision_score(Y_train, Y_train_pred_round), 3)
    pr_test = round(metrics.precision_score(Y_test, Y_test_pred_round), 3)
    rec_train = round(metrics.recall_score(Y_train, Y_train_pred_round), 3)
    rec_test = round(metrics.recall_score(Y_test, Y_test_pred_round), 3)
    if silent == False:
        print(f'''Train AUC score: {auc_train}, Train KS-test p-value: {round(ks_train.pvalue, 3)}, 
              Train F1-score: {f1_train}, Train precision: {pr_train}, Train recall: {rec_train}''')
        print(f'''Test AUC score: {auc_test}, Test KS-test p-value: {round(ks_test.pvalue, 3)}, 
              Test F1-score: {f1_test}, Test precision: {pr_test}, Test recall: {rec_test}''')
        print(results.summary())

    return results, auc_train, auc_test, round(ks_train.pvalue, 9), round(ks_test.pvalue, 9),\
           f1_train, f1_test, pr_train, pr_test, rec_train, rec_test

### Modelling

In [21]:
# Read dataset and define columns for feature generation
data = pd.read_parquet('Data/dataset.parquet')
indices = data.groupby(['Ticker', 'Index']).size().index.values
cols = ['Hurst', 'Correlation Dimension', 'Lyapunov', 
        'Variance', 'Skewness', 'Kurtosis', 'PSD', 'ACF_1',
        'WL_C1', 'WL_C2', 'WL_C3']

# Set lag for dynamics and short variance calculation
lag_model = [8]

# Calculate dynamics and short variance
# Original idea about variance was born from the largest Lyapunov exponent's behaviour before the critical transition point:
# is mostly didn't move in nominal values but its variance in some cases decreased signigicantly 
data_logdyn = pd.DataFrame()
for ind in tqdm(indices):
    data_ind = data[(data['Ticker'] == ind[0]) & (data['Index'] == ind[1])]
    for col in cols:
        for lag_m in lag_model:
            data_ind[col + '_' + str(lag_m) + '_dyn'] = data_ind[col] / data_ind[col].shift(lag_m) - 1
            data_ind[col + '_' + str(lag_m) + '_Variance'] = data_ind[col].rolling(lag_m).var()
    data_ind.dropna(inplace = True)
    data_logdyn = pd.concat([data_logdyn, data_ind])

# Reset index to get rid of dates and save final dataset
data_logdyn.reset_index(drop = True, inplace = True)
data_logdyn = data_logdyn[data_logdyn['Distance'] > 0]
data_logdyn.to_parquet('Data/final_dataset.parquet')
data_logdyn

100%|██████████| 976/976 [00:47<00:00, 20.47it/s]


,Volume,MA100,Rise,Distance,Index,Ticker,Hurst,Correlation Dimension,Lyapunov,Variance,...,PSD_8_dyn,PSD_8_Variance,ACF_1_8_dyn,ACF_1_8_Variance,WL_C1_8_dyn,WL_C1_8_Variance,WL_C2_8_dyn,WL_C2_8_Variance,WL_C3_8_dyn,WL_C3_8_Variance
0,382075.0,186099.90,False,291,2175,A,0.651958,-2.785248e-15,0.005159,9.550465e+09,...,0.005967,0.000014,-0.030671,0.000010,0.077689,0.000880,-8.733268,0.000429,-1.455061,0.000175
1,292647.0,187908.86,False,290,2175,A,0.613662,1.071124e-15,0.001808,9.585082e+09,...,0.005660,0.000015,-0.006026,0.000011,-0.043911,0.000973,1.643379,0.000485,-0.750029,0.000142
2,77650.0,186442.90,False,289,2175,A,0.692141,-2.569802e-15,0.004011,9.586808e+09,...,0.007798,0.000015,-0.012033,0.000012,0.030140,0.001035,-3.108573,0.000570,1.116842,0.000160
3,69826.0,185885.98,False,288,2175,A,0.691744,1.381300e-15,0.003034,9.594264e+09,...,0.015778,0.000009,-0.009283,0.000012,0.098330,0.000763,-0.748113,0.000410,0.362201,0.000187
4,68277.0,185620.89,False,287,2175,A,0.815697,7.129526e-16,0.001907,9.605297e+09,...,0.017787,0.000005,-0.009602,0.000010,-0.021040,0.000724,-6.134960,0.000442,-1.047026,0.000129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284986,305417.0,163643.22,False,5,2923,ZWS,0.629390,9.868228e-16,0.000374,9.862010e+09,...,-0.105693,0.000085,0.054352,0.000184,-0.065463,0.000673,-0.651608,0.002305,-0.487456,0.000051
284987,208657.0,164804.55,False,4,2923,ZWS,0.650464,6.793770e-16,0.002001,9.805879e+09,...,-0.105187,0.000075,0.044414,0.000244,0.013370,0.000620,-0.092599,0.002211,-0.182851,0.000050
284988,1369475.0,177527.24,False,3,2923,ZWS,0.592440,8.986417e-16,0.006145,1.290870e+10,...,-0.092668,0.000062,-0.171816,0.001229,0.033572,0.000754,-0.400974,0.002288,0.036566,0.000054
284989,680336.0,183571.70,False,2,2923,ZWS,0.623110,-1.746746e-15,0.006337,1.351842e+10,...,-0.089003,0.000046,0.009844,0.001233,0.040718,0.000743,-0.075623,0.002146,-0.270012,0.000061


In the cell below we are iterating over the three lists of parameters:
- horizons - how many hours before the transition are considered to be close enough to be prediction phase
- sizes - share of the positive observations in the whole modelling dataset - this parameter is important because in the original dataset share of positives for some of the horizons was to small, so we dicided to use decrease size of the negative dataset and randomize it
- states - in order to avoid lucky random choices in the sizes randomization we are use a list of different random states to average the results

In [22]:
# Read dataset
data_logdyn = pd.read_parquet('Data/final_dataset.parquet')

# Choose binary target and other parameters
target = 'Flag'
horizons = list(range(2, 13))
shares = np.linspace(0.05, 0.2, 4)
states = list(range(0, 10000, 500))

# Create dataframe for the results
res = pd.DataFrame(columns = ['Horizon', '1 Share', '1 Share real', 'State',
                              'Train size', 'Test size', 'Train AUC', 'Test AUC',
                              'Train KS-test p-value', 'Test KS-test p-value',
                              'Train F1-score', 'Test F1-score', 
                              'Train precision', 'Test precision', 
                              'Train recall', 'Test recall', 'Variables'])

# Iterate over the chosen parameters and optimize classification models, then save all the results to the dataframe
for horizon in tqdm(horizons):
    data_testing = data_logdyn.copy()
    data_testing['Flag'] = data_testing['Distance'].apply(lambda x: 0 if x > horizon else 1)
    data_testing.drop(columns = ['Volume', 'MA100', 'Rise', 'Distance', 'Index', 'Ticker'], inplace = True)
    
    data_testing_1 = data_testing[data_testing[target] == 1]
    data_testing_0 = data_testing[data_testing[target] == 0]
    Y_1 = data_testing_1[target]
    X_1 = data_testing_1.drop(columns = [target])
    share_1_orig = len(data_testing_1) / (len(data_testing_0) + len(data_testing_1))
    for share in shares:
        for state in states:
            _, X_0, _, Y_0 = modsel.train_test_split(data_testing_0.drop(columns = [target]), data_testing_0[target], 
                                                     test_size = min(share_1_orig * (1 - share) / share, 1), random_state = state)
            share_1 = len(Y_1) / (len(Y_0) + len(Y_1))
            Y = pd.concat([Y_0, Y_1])
            X = sm.add_constant(pd.concat([X_0, X_1]))
            X_train, X_test, Y_train, Y_test = modsel.train_test_split(X, Y, test_size = 0.2, random_state = state)
            results_rs, auc_train_rs, auc_test_rs, ks_train_rs, ks_test_rs, f1_train_rs,\
                f1_test_rs, pr_train_rs, pr_test_rs, rec_train_rs, rec_test_rs\
                = model_optimization(Y_train, Y_test, X_train, X_test, silent = True)
            res.loc[len(res)] = [horizon, share, share_1, state, len(Y_train), len(Y_test),
                                 auc_train_rs, auc_test_rs, ks_train_rs, ks_test_rs,
                                 f1_train_rs, f1_test_rs, pr_train_rs, pr_test_rs,
                                 rec_train_rs, rec_test_rs, list(results_rs.params.index)]

# OHE-like transformation of the variables' lists
res_counts = res['Variables'].to_frame()
for col in list(X_1.columns) + ['const']:
    res_counts[col] = res_counts['Variables'].apply(lambda x: 1 if col in x else 0)
res = res.drop(columns = ['Variables']).join(res_counts.drop(columns = ['Variables']))
res.to_parquet('Data/params.parquet')

# Create pivot based on the horizon and 1 share parameters
groups = ['Horizon', '1 Share', '1 Share real']
drops = ['State']
res_means = res.groupby(groups)[res.columns.drop(groups + drops)].mean()
res_means.to_parquet('Data/params_mean.parquet')
res_means

100%|██████████| 11/11 [2:04:38<00:00, 679.91s/it] 


Train size  Test size  Train AUC  Test AUC  \
Horizon 1 Share 1 Share real                                               
2       0.05    0.050327         31028.0     7758.0    0.88715   0.88535   
        0.10    0.100619         15520.0     3880.0    0.87780   0.87215   
        0.15    0.150873         10350.0     2588.0    0.86815   0.87015   
        0.20    0.201092          7765.0     1942.0    0.86445   0.85880   
3       0.05    0.050494         46389.0    11598.0    0.84960   0.84255   
        0.10    0.100934         23207.0     5802.0    0.84215   0.84020   
        0.15    0.151326         15479.0     3870.0    0.84895   0.84660   
        0.20    0.201653         11616.0     2904.0    0.84370   0.84395   
4       0.05    0.050661         61648.0    15413.0    0.80620   0.80500   
        0.10    0.101250         30846.0     7712.0    0.81070   0.80830   
        0.15    0.151771         20578.0     5145.0    0.80190   0.80135   
        0.20    0.202217         15444.0     3862.0    0.80190   0.80060   
5       0.05    0.050830         76805.0    19202.0    0.77270   0.76760   
        0.10    0.101569         38436.0     9610.0    0.77280   0.77125   
        0.15    0.152219         25647.0     6412.0    0.77300   0.76740   
        0.20    0.202784         19252.0     4813.0    0.77005   0.76810   
6       0.05    0.050999         91860.0    22966.0    0.74610   0.74335   
        0.10    0.101890         45979.0    11495.0    0.74720   0.74390   
        0.15    0.152675         30684.0     7672.0    0.74785   0.74535   
        0.20    0.203347         23038.0     5760.0    0.74610   0.73975   
7       0.05    0.051169        106814.0    26704.0    0.72690   0.72305   
        0.10    0.102213         53472.0    13369.0    0.72665   0.73020   
        0.15    0.153129         35692.0     8924.0    0.72745   0.73025   
        0.20    0.203922         26802.0     6701.0    0.72865   0.72900   
8       0.05    0.051341        121665.0    30417.0    0.70770   0.70550   
        0.10    0.102536         60919.0    15230.0    0.71115   0.71005   
        0.15    0.153589         40669.0    10168.0    0.71000   0.71125   
        0.20    0.204494         30545.0     7637.0    0.71115   0.71215   
9       0.05    0.051513        136415.0    34104.0    0.69560   0.69480   
        0.10    0.102863         68316.0    17079.0    0.69665   0.69460   
        0.15    0.154049         45616.0    11405.0    0.69045   0.68810   
        0.20    0.205071         34267.0     8567.0    0.69425   0.69180   
10      0.05    0.051687        151062.0    37766.0    0.68265   0.68145   
        0.10    0.103191         75665.0    18917.0    0.68415   0.68215   
        0.15    0.154511         50533.0    12634.0    0.68500   0.68130   
        0.20    0.205651         37967.0     9492.0    0.68530   0.68380   
11      0.05    0.051862        165608.0    41402.0    0.67265   0.66995   
        0.10    0.103521         82966.0    20742.0    0.67155   0.67015   
        0.15    0.154979         55419.0    13855.0    0.67160   0.66750   
        0.20    0.206235         41645.0    10412.0    0.66380   0.66335   
12      0.05    0.052039        180051.0    45013.0    0.66270   0.66415   
        0.10    0.103854         90219.0    22555.0    0.66370   0.66370   
        0.15    0.155447         60275.0    15069.0    0.66125   0.65710   
        0.20    0.206820         45303.0    11326.0    0.65920   0.65740   

                              Train KS-test p-value  Test KS-test p-value  \
Horizon 1 Share 1 Share real                                                
2       0.05    0.050327                        0.0                   0.0   
        0.10    0.100619                        0.0                   0.0   
        0.15    0.150873                        0.0                   0.0   
        0.20    0.201092                        0.0                   0.0   
3       0.05    0.050494                        0.0                   0

In [25]:
# Get mean metrics for all of the columns to understand what variables are actually used in the final models
round(res_means[np.in1d(res_means.index.get_level_values(0), list(range(4,9)))].mean(), 2)
# round(res_means.mean(), 2)

Train size                          47839.75
Test size                           11960.60
Train AUC                               0.75
Test AUC                                0.75
Train KS-test p-value                   0.00
Test KS-test p-value                    0.00
Train F1-score                          0.42
Test F1-score                           0.42
Train precision                         0.35
Test precision                          0.36
Train recall                            0.57
Test recall                             0.57
Hurst                                   0.03
Correlation Dimension                   0.00
Lyapunov                                0.44
Variance                                0.00
Skewness                                0.96
Kurtosis                                0.97
PSD                                     0.98
ACF_1                                   0.09
WL_C1                                   0.04
WL_C2                                   0.04
WL_C3     

In [24]:
# Vizual check of the single model
horizon = 8
share = 0.1
state = 2000
data_testing = data_logdyn.copy()
data_testing['Flag'] = data_testing['Distance'].apply(lambda x: 0 if x >= horizon else 1)
data_testing.drop(columns = ['Volume', 'MA100', 'Rise', 'Distance', 'Index', 'Ticker'], inplace = True)

data_testing_1 = data_testing[data_testing[target] == 1]
data_testing_0 = data_testing[data_testing[target] == 0]
Y_1 = data_testing_1[target]
X_1 = data_testing_1.drop(columns = [target])
_, X_0, _, Y_0 = modsel.train_test_split(data_testing_0.drop(columns = [target]), data_testing_0[target], 
                                                     test_size = min(share_1_orig * (1 - share) / share, 1), random_state = state)
share_1 = len(Y_1) / (len(Y_0) + len(Y_1))
Y = pd.concat([Y_0, Y_1])
X = sm.add_constant(pd.concat([X_0, X_1]))
X_train, X_test, Y_train, Y_test = modsel.train_test_split(X, Y, test_size = 0.2, random_state = state)
results_rs, auc_train_rs, auc_test_rs, ks_train_rs, ks_test_rs, f1_train_rs,\
    f1_test_rs, pr_train_rs, pr_test_rs, rec_train_rs, rec_test_rs\
    = model_optimization(Y_train, Y_test, X_train, X_test, silent = True)
print(results_rs.summary())
Y_test_pred = results_rs.predict(X_test)
ks_samples = pd.DataFrame({'Y': Y_test, 'Y_pred': Y_test_pred})
ks_samples_posi = ks_samples[ks_samples['Y'] == 1]['Y_pred']
ks_samples_nega = ks_samples[ks_samples['Y'] == 0]['Y_pred']
fig = go.Figure()
fig.add_trace(go.Histogram(x = ks_samples_posi, name = 'Posi'))
fig.add_trace(go.Histogram(x = ks_samples_nega, name = 'Nega'))
fig.update_layout(barmode = 'overlay')
fig.update_traces(opacity = 0.75)
fig.show()

                          Probit Regression Results                           
Dep. Variable:                   Flag   No. Observations:                87764
Model:                         Probit   Df Residuals:                    87748
Method:                           MLE   Df Model:                           15
Date:                Wed, 21 Feb 2024   Pseudo R-squ.:                  0.1400
Time:                        23:20:28   Log-Likelihood:                -17532.
converged:                       True   LL-Null:                       -20385.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.6583      0.051    -32.304      0.000      -1.759      -1.558
Skewness                             0.0734      0.024      3.025